# 02 - Data from the Web


In this homework we will extract interesting information from www.topuniversities.com and www.timeshighereducation.com, two platforms that maintain a global ranking of worldwide universities. This ranking is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need! You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. For this task, Postman with the Interceptor extension can help you greatly. We recommend that you watch this brief tutorial to understand quickly how to use it.

In [4]:
from bs4 import BeautifulSoup
import requests
import json
import string
import pandas as pd
import re

## Data from QS ranking

Multiple strategies have been testes in order to find the data used to construct the table of the QS Rankings. However, only one of them led to the expected result. We opened the Web Inspector on the website. Among the ressources used by the page, a text file named 'XHR/357051.txt'  was containing the expected ranking with the desired informations. 

In [5]:
URL_QS = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508497791260'
r = requests.get(URL_QS)
page_body = r.json()
qs_df = pd.DataFrame(page_body['data'])

In [39]:
qs_df.to_csv('test2.csv')

## Test to scrap additional data

In [14]:
URL_ADD = 'https://www.topuniversities.com/universities/stanford-university'
r = requests.get(URL_ADD)
page_body = r.text
soup = BeautifulSoup(page_body, 'html.parser')
row = soup.find('div', {'class' : 'total faculty', 'class' : 'number'}).text
number = int(re.sub("[^\\d]",'',row)) #Je pense qu'il sera bien de regarder un peu pourquoi ce 'int' à rajouter. Si c'est unq uestion de version ou pas

In [15]:
number

4285

## Data from Times Higher Education ranking

In [49]:
URL_THE = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'
r = requests.get(URL_THE)
page_body = r.json()
the_df = pd.DataFrame(page_body['data'])

In [50]:
the_df;

### Clean the data

In [51]:
#we keep only the first 200

In [54]:
the_df = the_df[:200]

In [55]:
the_df;

,aliases,location,member_level,name,nid,rank,rank_order,record_type,scores_citations,scores_citations_rank,...,scores_research,scores_research_rank,scores_teaching,scores_teaching_rank,stats_female_male_ratio,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio,subjects_offered,url
0,University of Oxford,United Kingdom,0,University of Oxford,468,1,10,master_account,99.1,15,...,99.5,1,86.7,5,46 : 54,"20,409",38%,11.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-oxford
1,University of Cambridge,United Kingdom,0,University of Cambridge,470,2,20,master_account,97.5,29,...,97.8,3,87.8,3,45 : 55,"18,389",35%,10.9,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/university-cambridge
2,California Institute of Technology caltech,United States,0,California Institute of Technology,128779,=3,30,private,99.5,10,...,97.5,4,90.3,1,31 : 69,"2,209",27%,6.5,"Architecture,Biological Sciences,Business & Ma...",/world-university-rankings/california-institut...
3,Stanford University,United States,11,Stanford University,467,=3,40,private,99.9,4,...,96.7,5,89.1,2,42 : 58,"15,845",22%,7.5,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/stanford-university
4,Massachusetts Institute of Technology,United States,0,Massachusetts Institute of Technology,471,5,50,private,100.0,1,...,91.9,9,87.3,4,37 : 63,"11,177",34%,8.7,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/massachusetts-insti...
5,Harvard University,United States,0,Harvard University,466,6,60,private,99.7,8,...,98.4,2,84.2,9,None,"20,326",26%,8.9,"Agriculture & Forestry,Archaeology,Art, Perfor...",/world-university-rankings/harvard-university
6,Princeton University,United States,0,Princeton University,469,7,70,private,99.6,9,...,93.9,6,85.7,7,45 : 55,"7,955",24%,8.3,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/princeton-university
7,Imperial College London,United Kingdom,0,Imperial College London,472,8,80,master_account,96.7,40,...,88.7,12,81.7,13,37 : 63,"15,857",55%,11.4,"Biological Sciences,Chemical Engineering,Chemi...",/world-university-rankings/imperial-college-lo...
8,University of Chicago,United States,0,University of Chicago,473,9,90,private,99.4,12,...,90.1,10,85.3,8,44 : 56,"13,525",25%,6.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-chicago
9,ETH Zurich – Swiss Federal Institute of Techno...,Switzerland,0,ETH Zurich – Swiss Federal Institute of Techno...,479,=10,100,master_account,94.3,60,...,92.0,8,76.4,21,31 : 69,"19,233",38%,14.6,"Agriculture & Forestry,Architecture,Biological...",/world-university-rankings/eth-zurich-swiss-fe...


## Quelles données nous sont inutiles? 

Je propose de garder: 
- name
- location
- nid (TBD)
- rank
- stat female: male (TBD)
- stats number students
- stats % intl students (to rename and maybe change the percentage by a number)
- stat student staff ratio


In [63]:
the_df = the_df.loc[ :, ['name', 'rank', 'location', 'nid', 'stats_female_male_ratio', 'stats_number_students', 'stats_pc_intl_students', 'stats_student_staff_ratio']]

In [64]:
the_df

,name,rank,location,nid,stats_female_male_ratio,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio
0,University of Oxford,1,United Kingdom,468,46 : 54,"20,409",38%,11.2
1,University of Cambridge,2,United Kingdom,470,45 : 55,"18,389",35%,10.9
2,California Institute of Technology,=3,United States,128779,31 : 69,"2,209",27%,6.5
3,Stanford University,=3,United States,467,42 : 58,"15,845",22%,7.5
4,Massachusetts Institute of Technology,5,United States,471,37 : 63,"11,177",34%,8.7
5,Harvard University,6,United States,466,None,"20,326",26%,8.9
6,Princeton University,7,United States,469,45 : 55,"7,955",24%,8.3
7,Imperial College London,8,United Kingdom,472,37 : 63,"15,857",55%,11.4
8,University of Chicago,9,United States,473,44 : 56,"13,525",25%,6.2
9,ETH Zurich – Swiss Federal Institute of Techno...,=10,Switzerland,479,31 : 69,"19,233",38%,14.6
